# Importing libraries and Data pre-processing 

In [ ]:
import cudf,cuml,cupy 
import pandas as pd
from cuml.feature_extraction.text import TfidfVectorizer as GPU_TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer as CPU_TfidfVectorizer
import time 
import seaborn as sns 
import matplotlib.pyplot as plt

# Import data
data1_url = "https://raw.githubusercontent.com/chris1610/pbpython/master/data/hospital_account_info.csv"
data2_url = "https://raw.githubusercontent.com/chris1610/pbpython/master/data/hospital_reimbursement.csv"

account = pd.read_csv(data1_url) #Hospital account information
reimbursement = pd.read_csv(data2_url) #Hospital reimbursement information

#Converting facility name, address, city and state into one string 
account_full_address = account.apply(lambda x: " ".join([x['Facility Name'],x['Address'],x['City'],x['State']]), axis=1).to_list() 

reimbursement_full_address = reimbursement.apply(lambda x: " ".join([x['Provider Name'],x['Provider Street Address'],x['Provider City'],x['Provider State']]), axis=1).to_list() 

In [ ]:
print(reimbursement.head(2).to_markdown())

# TF-IDF Vectorization - experimenting the effect of data size on computational time

In [ ]:
#Inititate sklearn vectoriser and cuml vectosier 

#CPU vectorizer from sklearn 
cpu_vectorizer = CPU_TfidfVectorizer(analyzer='char',ngram_range=(1,2))
#GPU vectorizer from cuml 
gpu_vectorizer = GPU_TfidfVectorizer(analyzer='char',ngram_range=(1,2))

#Here analyzer='char' means we are using character as the input 
#ngram_range = (1,2) means we are looking at both unigram and bigram for the model input 

In [ ]:
#Manually inflating number of rows with 10 run times 
total_datasize = []
cpu_time = []
gpu_time = [] 
for run in range(1,10):
  for i in range(1,10):
    #Manually inflating the number of records 
    input = reimbursement_full_address*i
    total_datasize.append(len(input))
    #Cpu runtime 
    start = time.time()
    cpu_output = cpu_vectorizer.fit_transform(input)
    done = time.time()
    elapsed = done - start 
    cpu_time.append(elapsed)

    #gpu runtime 
    start = time.time()
    #Convert input to cudf series 
    gpu_output = gpu_vectorizer.fit_transform(cudf.Series(input))
    done = time.time()
    elapsed = done - start 
    gpu_time.append(elapsed)

In [ ]:
import seaborn as sns 
fig, ax = plt.subplots(figsize=(10,10))
gpu_elapsed = pd.DataFrame({"time":gpu_time,"data_size":total_datasize,'label':"gpu"})
cpu_elapsed = pd.DataFrame({"time":cpu_time,"data_size":total_datasize,'label':"cpu"})
result = pd.concat([gpu_elapsed,cpu_elapsed]).reset_index()
sns.lineplot(x= 'data_size',y='time',hue = 'label',data = result,ax = ax )
plt.xlabel('Data Size')
plt.ylabel("Time Elapsed ")
plt.title("Comparing the speed of TF-IDF vectorisation on CPU and GPU")
plt.show()


# Consine similarity - experimenting the effect of matrix multiplication on computational time



In [ ]:
cpu_target =cpu_vectorizer.transform(account_full_address)
gpu_target = gpu_vectorizer.transform(cudf.Series(account_full_address))

In [ ]:
import cupy as cp
import numpy as np 
import scipy
import gc
import torch 
from cupyx.scipy.sparse.linalg import norm as cp_norm
from scipy.sparse.linalg import norm as sc_norm 

def np_cosine_similarity(query, target):
    # Assert that the input matrices have the same number of columns
    assert(query.shape[1] == target.shape[1])

    #Calculate the dot product 
    dot_product = np.dot(query,target.T)

    #Calculate l2 norm for query and target 
    query_norm = sc_norm(query,axis=1)
    target_norm = sc_norm(target,axis=1)

    return dot_product/(query_norm[:,np.newaxis]*target_norm[:,np.newaxis].T)

#Cupy is a drop-in replacement for numpy 
def cp_cosine_similarity(query, target):
    # Assert that the input matrices have the same number of columns
    assert(query.shape[1] == target.shape[1])
    #Initiate GPU instance 
    with cp.cuda.Device(0):
        #Create memory pool
        pool = cp.get_default_memory_pool()
        pool.set_limit(1e9)  # Set the limit of the memory pool to 1 GB
        
        #Check whether the sparse matrix is compatible with Cupy, if not then convert
        if isinstance(query,scipy.sparse._csr.csr_matrix) and isinstance(target,scipy.sparse._csr.csr_matrix):
        # Convert the input matrices to sparse format and copy to the GPU
            query = cp.sparse.csr_matrix(query, copy=True) 
            target = cp.sparse.csr_matrix(target, copy=True)
        
        # Dot product using cupy.dot()
        dot_product = query.dot(target.T)
        
        # Calculate l2 norm for query and target
        query_norm = cp_norm(query,axis=1)
        target_norm = cp_norm(target,axis=1)
        
        # Compute the cosine similarity
        output = dot_product / (cp.expand_dims(query_norm,axis=1) * cp.expand_dims(target_norm,axis=0))
        
        #Converting back to numpy array
        result = output.get()
    return result





In [ ]:
from numba import cuda 
#Is a JIT compiler for python
#You can take function with numerical values or arrays
#And compile them to assembley, so they run at high speed 
#Deploy to compiled on GPU and CUDA 

#Define the kernel for calculation 
#- A GPU function launched by the host and executed on the device
#Cannot explicity return a numerical value 
@cuda.jit
def pairwise_cosine_similarity(A, B, C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        dot = 0.0
        normA = 0.0
        normB = 0.0
        for k in range(A.shape[1]):
            a = A[i, k]
            b = B[j, k]
            dot += a * b
            normA += a ** 2
            normB += b ** 2
        C[i, j] = dot / (normA * normB)**0.5

def Numba_cuda_cosine_similarity(query,target): 
     # Assert that the input matrices have the same number of columns
    assert(query.shape[1] == target.shape[1])
    ## Allocate memory on the device for the result
    output = cuda.device_array((query.shape[0],target.shape[0]))


    # Convert the input matrices to sparse format and copy to the GPU
    query = cuda.to_device(query.toarray())
    target = cuda.to_device(target.toarray()) 
    #Set the number of threads in a block ----- 
    threadsperblock = (32,32)

    # Calculate the number of thread blocks in the grid 
    blockspergrid_x = (output.shape[0] + (threadsperblock[0] - 1)) // threadsperblock[0]
    blockspergrid_y = (output.shape[1] + (threadsperblock[1] - 1)) // threadsperblock[1]
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    #Starting the kernel 
    pairwise_cosine_similarity[blockspergrid, threadsperblock](query, target, output)

    # Copy the result back to the host
    return output.copy_to_host()


In [ ]:
from tqdm import tqdm 
total_datasize = []
cpu_time = []
gpu_time = [] 
numba_time = [] 

for size in tqdm(range(1000,cpu_output.shape[0],5000)):
    total_datasize.append(size)
    query = cpu_output[0:size,]
    #CPU --------------
    start = time.time()
    _ = np_cosine_similarity(query,cpu_target)
    done = time.time()
    elapsed = done - start 
    cpu_time.append(elapsed)
    
    #GPU --------------
    start = time.time()
    _ = cp_cosine_similarity(query,cpu_target)
    done = time.time()
    elapsed = done - start 
    gpu_time.append(elapsed)

    #Numba CUDA ---- 
    start = time.time()
    _ = Numba_cuda_cosine_similarity(query,cpu_target)
    done = time.time()
    elapsed = done - start 
    numba_time.append(elapsed) 


In [ ]:
import seaborn as sns 
fig, ax = plt.subplots(figsize=(10,10))
gpu_elapsed = pd.DataFrame({"time":gpu_time,"data_size":total_datasize,'label':"gpu"})
cpu_elapsed = pd.DataFrame({"time":cpu_time,"data_size":total_datasize,'label':"cpu"})
numba_elasped = pd.DataFrame({"time":numba_time,"data_size":total_datasize,'label':"numba"}) 
result = pd.concat([gpu_elapsed,cpu_elapsed,numba_elasped]).reset_index()
sns.lineplot(x= 'data_size',y='time',hue = 'label',data = result,ax = ax )
plt.xlabel('Data Size')
plt.ylabel("Time Elapsed ")
plt.title("Comparing the speed of matrix multiplication on CPU and GPU")
plt.show()


# Sanity check  

In [ ]:
from collections import defaultdict 
#Full - pipeline on GPU 
reimbursement_tfidf= gpu_vectorizer.fit_transform(cudf.Series(reimbursement_full_address))
account_tfidf = gpu_vectorizer.transform(cudf.Series(account_full_address))
similarity_matrix = Numba_cuda_cosine_similarity(reimbursement_tfidf,account_tfidf) 

result = defaultdict(list)
#Getting the reimbursement address and state, account address and state 
for index in range(similarity_matrix.shape[0]):
    most_similar_index = similarity_matrix[index].argmax()
    result['reimbursement_address'].append(reimbursement_full_address[index]) 
    result['account_address'].append(account_full_address[most_similar_index])  
    result['reimbursment_zip'].append(reimbursement.loc[index,'Provider Zip Code'])
    result['account_zip'].append(account.loc[most_similar_index,'ZIP Code']) 
    result['similarity_score'].append(similarity_matrix[index][most_similar_index])

result_df = pd.DataFrame(result).sort_values('similarity_score')

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(result_df.reimbursment_zip,result_df.account_zip)

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots(figsize=(10,10))
result_df.similarity_score.plot(kind='hist',bins=10,ax=ax)
plt.xlabel('Cosine Similarity Score')
plt.title('Distribution of cosine similarity score')

In [ ]:
print(result_df.sort_values('similarity_score',ascending=True).head(2).to_markdown())

In [ ]:
print(result_df.sort_values('similarity_score',ascending=False).head(2).to_markdown())